## SECTION1: 계층적 색인
* 다중(둘 이상) 색인 단계를 지정할 수 있도록 함
* 추상적으로 말하면, 높은 차원의 데이터를 낮은 차원의 형식으로 다룰 수 있게 해주는 기능

In [1]:
import pandas as pd
import numpy as np

In [2]:
# MultiIndex를 색인으로 하는 Series
data = pd.Series(np.random.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [3]:
data

a  1   -1.071285
   2   -0.777589
   3    0.507701
b  1   -0.800641
   3    0.171801
c  1   -0.221812
   2    2.628213
d  2    0.130152
   3   -1.158018
dtype: float64

In [4]:
# 색인의 계층을 보여줌
# 위 단계의 색인을 이용해서 하위 계층을 직접 접근할 수 있음
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
# 계층적으로 색인된 객체는 데이터의 부분집합을 부분적 색인으로 접근하는 것이 가능
# 이를 partial indexing이라고 함
data['b']

1   -0.800641
3    0.171801
dtype: float64

In [6]:
# 연속 선택
data['b':'c']

b  1   -0.800641
   3    0.171801
c  1   -0.221812
   2    2.628213
dtype: float64

In [7]:
# 부분 선택
data.loc[['b', 'd']]

b  1   -0.800641
   3    0.171801
d  2    0.130152
   3   -1.158018
dtype: float64

In [8]:
# 연속 선택 확인용
data['a':'c']

a  1   -1.071285
   2   -0.777589
   3    0.507701
b  1   -0.800641
   3    0.171801
c  1   -0.221812
   2    2.628213
dtype: float64

In [9]:
# 부분 선택 확인용
data.loc[['b', 'd', 'a']]

a  1   -1.071285
   2   -0.777589
   3    0.507701
b  1   -0.800641
   3    0.171801
d  2    0.130152
   3   -1.158018
dtype: float64

In [10]:
# 하위 계층의 객체를 선택하는 것도 가능
data.loc[:, 2]

a   -0.777589
c    2.628213
d    0.130152
dtype: float64

In [11]:
# 계층적 색인은 데이터를 재형성하고 피벗테이블 생성 같은 그룹 기반의 작업을 할 때 중요하게 사용
# DataFrame 객체에 unstack 메서드를 사용해서 데이터를 새롭게 배열
# 오호! 계층형 인덱스를 가진 Series 객체에서 2차원 DataFrame이 되었군룡
data.unstack()

,1,2,3
a,-1.071285,-0.777589,0.507701
b,-0.800641,NaN,0.171801
c,-0.221812,2.628213,NaN
d,NaN,0.130152,-1.158018


In [12]:
# unstack 반대작업은 stack
# 원본 파일을 변경하지 않고 값을 반환해주는 형태의 메서드 unstack, stack
data.unstack().stack()

a  1   -1.071285
   2   -0.777589
   3    0.507701
b  1   -0.800641
   3    0.171801
c  1   -0.221812
   2    2.628213
d  2    0.130152
   3   -1.158018
dtype: float64

In [14]:
# DataFrame에서는 두 축 모두 계층적 색인을 가질 수 있음
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

In [15]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [16]:
# 계층적 색인의 각 단계는 이름을 가질 수 있음
# 만약 이름을 가지고 있다면 콘솔 출력시 함께 나타남
frame.index.names = ['key1', 'key2']

In [17]:
frame.columns.names = ['state', 'color']

In [18]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [19]:
# 컬럼의 부분집합을 부분적인 색인으로 접근하는 것도 컬럼에 대한 부분적 색인과 비슷하게 사용
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [22]:
MultiIndex = data.index

In [23]:
# MultiIndex는 따로 생성한 다음 재사용 가능
# DataFrame의 컬럼 계층 이름은 다음처럼 생성할 수 있음
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### MARK: 계층 순서 변경 및 정렬

In [25]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [24]:
# swaplevel 메서드: 넘겨받은 두 개의 계층 번호나 이름이 뒤바뀐 새로운 객체 반환 (데이터는 변경되지 않음)
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [26]:
# sort_index 메서드: 단일 계층에 속한 데이터 정렬
# swaplevel을 이용해서 계층을 바꿀 때, sort_index를 이용하여 결과가 사전적으로 정렬되도록 만듬
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [27]:
# level=0: key1 기준, level=1: key2기준
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [28]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

<pre> * 객체가 계층적 색인으로 상위계층부터 사전식으로 정렬되어 있다면 데이터를 선택하는 성능이 훨씬 좋아진다! </pre>
### MARK: 계층별 요약 통계
* level 옵션을 가지고 있는데 어떤 한 축에 대해 합을 구하고 싶은 단계를 지정할 수 있는 옵션

In [29]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [30]:
# DataFrame에서 로우나 컬럼을 아래처럼 계층별로 합할 수 있다
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [31]:
frame.sum(level='color', axis=1) # axis는 뭘 의미하지

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [33]:
# default 값이 axis=0 즉, 인덱스를 기준으로 넘겨받은 level 이름 값을 찾아 통계값 도출
# 즉, axis=1 옵션이 없으면 'color' 값은 인덱스에 존재하지 않는 이름이기 때문에 에러 발생!
frame.sum(level='key1', axis=0)

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

### MARK: DataFrame 컬럼 사용
* 색인으로 하나 이상의 컬럼 사용
* 로우의 색인을 DataFrame의 컬럼으로 옮기고 싶을 때! 아래 예제를 살펴보자

In [34]:
frame = pd.DataFrame({'a': range(7), 
                      'b': range(7, 0, -1), 
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

In [35]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [36]:
# set_index 메서드: 하나 이상의 컬럼을 색인으로 하는 새로운 DataFrame 생성
# 원래 컬럼에 있던 값들을 Index로 가져와, 그 값을 기준으로 재배열
frame2 = frame.set_index(['c', 'd'])

In [37]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [38]:
# drop=False 옵션을 넘겨 컬럼을 명시적으로 남겨두지 않으면 DataFrame에서 삭제된다 (위 예제처럼)
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [39]:
# reset_index(): set_index()와 반대 개념, 계층적 색인 단계가 컬럼으로 이동
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## SECTION2: 데이터 합치기
<pre> pandas 객체에 저장된 데이터를 합치는 방법
 * 관계형 데이터베이스의 join 연산과 유사한 "pandas.merge"
 * 하나의 축을 따라 객체를 이어 붙이는 "pandas.concat"
 * 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체의 값으로 채우는 "combine_first"
</pre>

### MARK: pandas의 merge 함수
* 병합이나 조인 연산은 관계형 데이터베이스의 핵심적 연산
* 하나 이상의 "키"를 사용해서 데이터 집합의 로우를 합친다

In [40]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

In [41]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [42]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [43]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [44]:
# 다대일 경우의 예제의 merge 함수 호출 결과:
# df1의 데이터는 key 컬럼에 여러 개의 a, b 로우를 가지고 있음
# df2의 데이터는 key 컬럼에 유일한 로우를 가지고 있음
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [46]:
# merge함수는 크게 명시하지 않아도 중복된 컬럼 이름을 키로 사용
# 하지만 명시적으로 지정해주는 습관을 들이는 게 좋음
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [47]:
# 만약 두 객체에 중복된 이름이 하나도 없다면 따로 지정
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

In [48]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [49]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


* 결과를 잘 살펴보면 'c'와 'd'에 해당하는 값이 빠진 것을 알 수 있다.
* merge 함수는 기본적으로 "내부 조인"을 수행하여 교집합 결과를 반환하기 때문!
<pre> [how 인자로 <b>'left', 'right', 'outer'</b>을 넘겨 왼쪽 조인, 오른쪽 조인, 외부 조인을 수행할 수 있다]
 * how 인자의 default값: inner(내부 조인)
 * 외부 조인은 합집합 결과, 왼쪽 조인과 오른쪽 조인은 각각 왼쪽 또는 오른쪽의 모든 로우를 포함하는 결과를 반환한다!
</pre>

In [50]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [51]:
# 다대다 병합의 경우 잘 정의되어 있긴 하지만 직관적이지 않음
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

In [52]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})

In [53]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [54]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [55]:
# df1의 모든 키 조합 사용
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [56]:
# 다대다 조인의 경우 두 로우의 데카르트 곱 반환
# 조인 메서드는 결과에 나타나는 구별되는 키에 대해서만 적용
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [57]:
# 여러 개의 키를 병합하려면 컬럼 이름이 담긴 리스트를 넘기면 된다
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})

In [58]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [59]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


* merge 메서드의 종류에 따라 어떤 키 조합이 결과로 반환되는지 알려면 실제 구현과는 조금 다르지만
* 여러 개의 키가 들어 있는 튜플의 배열이 단일 조인키로 사용된다고 생각하면 된다.

In [60]:
# 병합 연산에서 고려해야 할 마지막 사항은 겹치는 컬럼 이름에 대한 처리
# 축 이름을 변경해서 수동으로 컬럼 이름이 겹치게 할 수 있음
# merge 함수에 있는 suffixes 인자로 두 DataFrame 객체에서 겹치는 컬럼 이름 뒤에 붙일 문자열을 지정해줄 수도 있음
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [61]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### MARK: 색인 병합하기
* 병합하려는 키가 DataFrame의 색인일 경우
* <b>left_index=True</b> 혹은 <b>right_index=True</b> 옵션(혹은 둘 다)을 지정해서 해당 색인을 병합키로 사용 가능

In [62]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})

In [63]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [64]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [65]:
right1

,group_val
a,3.5
b,7.0


In [66]:
# 왼쪽 리스트의 key 값을 기준으로 교집합
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [67]:
# 외부 조인을 실행해서 합집합을 구할 수도 있다
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [68]:
# 계층 색인된 데이터는 암묵적으로 여러 키를 병합하는 것이라 약간 복잡
# 여러 개의 컬럼을 지정하여 병합해야 됨
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 
                      'key2': [2000, 2001, 2002, 2001, 2002], 
                      'data': np.arange(5.)})

In [69]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [70]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [71]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [72]:
# 근데 교집합인데 왜 Ohio 결과가 4개 나오지?
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [73]:
# 중복된 색인값 다룰 때에는 how='outer' 옵션 사용
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [74]:
# 양 쪽에 공통적으로 존재하는 여러 개의 색인 병합 또한 가능
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])

In [75]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [76]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [77]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [78]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [80]:
# 색인으로 병합 시 join 메서드 활용하면 편리
# join 메서드는 컬럼이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [82]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [83]:
right1

,group_val
a,3.5
b,7.0


In [81]:
# pandas의 일부 코드 제약으로 인해 DataFrame의 join 메서드는 왼쪽 조인을 수행
# join 메서드를 호출한 DataFrame의 컬럼 중 하나에 대해 조인을 수행하는 것 또한 가능
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [84]:
# 색인 대 색인으로 두 Dataframe을 병합하려면 아주 간단!
# 병합하려는 DataFrame의 리스트를 join 메서드로 넘기면 가능
# 하지만 보통 이런 병합은 concat 메서드를 사용하는 것이 보편적
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [86]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [91]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [90]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [88]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [89]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### MARK: 축 따라 이어붙이기
* 데이터를 합치는 또 다른 방법으로 이어붙이기(concatenation)가 있다
* NumPy는 ndarray를 이어붙이는 concatenate 함수를 제공

In [92]:
arr = np.arange(12).reshape(3, 4)

In [93]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [94]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

pandas의 Series나 DataFrame의 경우, 객체의 컨텍스트 내부에는 축마다 이름이 있어 배열을 쉽게 이어붙일 수 있도록 되어 있다.
* 이 때 다음 사항을 고려해야한다
<pre> <b>고려사항</b>
 1. 만약 연결하려는 두 객체의 색인이 서로 다르면 결과는 색인의 교집합? 혹은 합집합?
 2. 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있어야 하는가?
 3. 어떤 축으로 연결할 것인지 고려해야 하는가? (많은 경우, 기본 정수 라벨이 가장 먼저 무시)
</pre>

In [96]:
# pandas의 concat 함수 실습
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [97]:
# concat함수는 axis=0을 기본값으로 함
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [99]:
# axis=1을 넘긴다면 결과는 Series가 아닌 DataFrame이 됨(axis=1은 컬럼을 의미)
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [100]:
# 겹치는 축이 없어 외부 조인으로 정렬된 합집합을 얻었으나, join='inner'를 넘겨 교집합을 구할 수 있음
s4 = pd.concat([s1, s3])

In [101]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [104]:
s1

a    0
b    1
dtype: int64

In [102]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [103]:
# 'f'와 'g' 라벨은 join='inner' 옵션으로 사라지게 됨
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [105]:
# join_axes 인자로 병합하려는 축을 지정해줄 수도 있음
# 엥 안되는디유... 왜쥬.... 그런 키워드 인자 안 받는다는디유....
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [106]:
# Series를 이어붙이기 전 개별 Series를 구분할 수 없는 문제 발생
# 이어붙인 축에 대해 계층적 색인을 생성하여 식별이 가능하도록 할 수 있음
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [107]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [108]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [109]:
# 계층적 색인을 생성하려면 keys 인자 사용
# Series를 axis=1로 병합할 경우 keys는 DataFrame의 컬럼 제목
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [110]:
# DataFrame 객체에 대해서도 지금"까지와 같은 방식으로 적용할 수 있음
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])

In [111]:
df2 = pd.DataFrame(5+np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])

In [112]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [113]:
df2

,three,four
a,5,6
c,7,8


In [114]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [115]:
# 리스트 대신 객체의 사전을 넘기면 사전의 키가 keys 옵션으로 사용
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [116]:
# 계층적 색인을 생성할 때 사용할 수 있는 추가적인 옵션 존재
# 아래는 names 인자로 새로 생성된 계층의 이름을 지정할 수 있음
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [117]:
# DataFrame의 로우 색인이 분석에 필요한 데이터를 포함하고 있지 않은 경우
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [119]:
df1

,a,b,c,d
0,0.446831,0.447602,-1.153491,-1.087702
1,-1.054701,-1.151160,1.753163,-0.890322
2,0.359741,0.145267,2.917572,1.342852


In [118]:
df2

,b,d,a
0,1.771512,0.555140,-0.284136
1,0.539930,-0.061936,0.202620


In [120]:
# 이 경우 ignore_index=True 옵션을 줌
# 그러면 기존 로우 색인은 무시하고, 새롭게 인덱스 넘버가 붙게 됨!
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.446831,0.447602,-1.153491,-1.087702
1,-1.054701,-1.151160,1.753163,-0.890322
2,0.359741,0.145267,2.917572,1.342852
3,-0.284136,1.771512,NaN,0.555140
4,0.202620,0.539930,NaN,-0.061936


In [121]:
pd.concat([df1, df2]) # 로우 색인이 기존 데이터 그대로 사용

,a,b,c,d
0,0.446831,0.447602,-1.153491,-1.087702
1,-1.054701,-1.151160,1.753163,-0.890322
2,0.359741,0.145267,2.917572,1.342852
0,-0.284136,1.771512,NaN,0.555140
1,0.202620,0.539930,NaN,-0.061936


### MARK: 겹치는 데이터 합치기
* 데이터 합칠 때 병합이나 이어붙이기로는 불가능한 상황 존재
  * = 두 데이터 셋의 색인이 일부 겹치거나 전체가 겹치는 경우
* 벡터화된 if-else 구문을 표현하는 NumPy의 where 함수 활용 예제를 살펴보자

In [122]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])

In [123]:
b[-1] = np.nan # 마지막 값 na 값으로 변경

In [124]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [125]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [126]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [127]:
# DataFrame의 combine_first(): 컬럼에 대해 같은 동작 수행
# 호출 객체에서 누락된 데이터를 인자로 넘긴 객체에 있는 값으로 채워넣을 수 있음
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

In [128]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [129]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [130]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [131]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## SECTION3: 재형성과 피벗
* 표 형식의 데이터를 재배치 (= 재형성/ 피벗 연산)
### MARK: 계층적 색인으로 재형성
<pre> <b>stack</b>
 * 데이터의 컬럼을 로우로 피벗(또는 회전)시킨다.
 * 즉, 컬럼-> 로우
</pre>
<pre> <b>unstack</b>
 * 로우를 컬럼으로 피벗시킨다.
 * 로우 -> 컬럼
</pre>

In [132]:
# 문자열이 담긴 배열을 로우와 컬럼의 색인아로 하는 작은 DataFrame
data = pd.DataFrame(np.arange(6).reshape((2, 3)), index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))

In [133]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [134]:
# stack 메서드를 사용하면 컬럼이 로우로 피벗되어 다음과 같은 Series 객체 반환 (계층적 색인 가지는 Series 객체)
# stack과 unstack은 기본적으로 가장 안쪽에 있는 레벨부터 끄집어 냄
result = data.stack()

In [135]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [136]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [137]:
# 레벨 숫자나 이름을 전달해서 끄집어낼 단계를 지정할 수 있음
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [138]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [139]:
# 해당 레벨에 있는 모든 값이 하위 그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터 발생 가능
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [140]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [141]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [142]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [143]:
# stack 메서드는 누락된 데이터를 자동으로 걸러내기 떄문에 연산을 쉽게 원상복구 할 수 있다
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [144]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [146]:
# DataFrame을 unstack()할 때, unstack() 레벨은 결과에서 가장 낮은 단계가 됨
df = pd.DataFrame({'lef1': result, 'right': result+5}, columns=pd.Index(['left', 'right'], name='side'))

In [147]:
df

side            left  right
state    number            
Ohio     one     NaN      5
         two     NaN      6
         three   NaN      7
Colorado one     NaN      8
         two     NaN      9
         three   NaN     10

In [148]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one     NaN      NaN     5        8
two     NaN      NaN     6        9
three   NaN      NaN     7       10

### MARK: 긴 형식에서 넓은 형식으로 피벗하기
* 데이터베이스나 csv 파일에 여러 개의 시계열 데이터를 저장하는 일반적인 방법: "시간 순서대로 나열"

In [150]:
data = pd.read_csv('examples/macrodata.csv')

In [151]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [156]:
# year 인자가 왜 없어요?
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='data')

AttributeError: 'DataFrame' object has no attribute 'year'

In [153]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [154]:
data = data.reindex(columns=columns)

In [ ]:
# periods가 생성이 안 돼서,,, 이 코드 실행 못 함다,,,
data.index = periods.to_timestamp('D', 'end')

In [ ]:
# 얘도.. 뒷부분 (~342쪽까지 ㅜㅜ)
ldata = data.stack().reset_index().rename(columns={0: 'value'})

### MARK: 넓은 형식에서 긴 형식으로 피벗하기
* pivot과 반대되는 연산 pandsas.melt
* 하나의 컬럼을 여러 개의 새로운 DataFrame으로 생성하기 보다는 여러 칼럼을 하나로 병합하고 입력보다 긴 형태로 만들어 냄
* 'key' 컬럼을 그룹 구분자로 사용할 수 있고 다른 컬럼을 데이터값으로 사용할 수 있다
* pandas.melt를 사용할 때는 반드시 어떤 컬럼을 그룹 구분자로 사용할 것인지 지정해야 한다.

In [157]:
# 'key'를 그룹 구분자로 지정
# ????? 아까부터 일 똑바로 안 하냐
melted = pd.melt(df, ['key'])

KeyError: "The following 'id_vars' are not present in the DataFrame: ['key']"

In [ ]:
melt